<a href="https://colab.research.google.com/github/yinon2592/DL_Project_046211/blob/main/section_c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
# drive.mount('/content/drive/my-drive/project_calculations')
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install transformers
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.2 MB/s eta 0:00:00


In [30]:
### Prepare data
import os
import pandas as pd
from torch.utils.data import Dataset
import re
import torch
# Step 1: Dataset Preparation
# Step 2: Data Preprocessing

# load section_c data (data already cleaned)
section_c_data_path = '/content/drive/My Drive/project_dataset/section_c_data.csv'
df = pd.read_csv(section_c_data_path)
df.dropna(how='any', inplace=True)
df = df.head(100)
print("dataset size is ", df.shape[0])
print(df.label.value_counts())
print(df.sample(5), "\n")

train_size = int(len(df) * 0.8)  # 80% for training
print("train_size is ", train_size)
val_size = len(df) - train_size  # remaining for validation
print("val_size is ", val_size)

train_dataset = df[:train_size]  # First train_size rows for training
val_dataset = df[train_size:train_size+val_size]  # Remaining val_size rows for validation

dataset size is  100
0    54
1    46
Name: label, dtype: int64
    label                                               text
19      0  beach this morning bbq this afternoon back to ...
89      1  thank you for reminding me i actually have som...
66      1                                    downtown disney
71      0                              need to charge my mac
94      0                     definitely just failed my exam 

train_size is  80
val_size is  20


In [45]:
from transformers import set_seed, GPT2Config, GPT2Tokenizer
import pandas as pd
from torch.utils.data import Dataset

model_config = GPT2Config.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')


tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.padding_side = "left" # Very Important
tokenizer.pad_token = tokenizer.eos_token

model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = model.config.eos_token_id

class Texts(Dataset):
    def __init__(self,dataset, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.texts = []

        for _, row in df.iterrows():
          label = row['label']
          sentiment = 'positive' if label == 1 else 'negative'
          text = row['text'].split()
          text.append(f". among positive and negative, the previous text was {sentiment}. ")
          self.texts.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))
        if truncate:
            self.texts = self.texts[:20000]
        self.texts_count = len(self.texts)

    def __len__(self):
        return self.texts_count

    def __getitem__(self, item):
        return self.texts[item]


train_dataset = Texts(control_code="<|startoftext|>", dataset = train_dataset, truncate=False, gpt2_type="gpt2")

class TweetDataset(Dataset):
    def __init__(self, dataset):
        super().__init__()
        self.data = dataset

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        record = self.data.iloc[index]
        text = record['text']
        return {'text': text, 'label': record['label']}


val_dataset = df[train_size:train_size+val_size].copy()  # Copy the slice of data
val_dataset = pd.DataFrame(val_dataset)
val_dataset = TweetDataset(dataset=val_dataset)

class Gpt2ClassificationCollator(object):
    def __init__(self, tokenizer, max_seq_len=None):
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len

        return

    def __call__(self, sequences):
        texts = [str(sequence['text']) for sequence in sequences]
        # print(texts)
        labels = [int(sequence['label']) for sequence in sequences]
        # labels = ["positive" if sequence['label']== 1 else "negative" for sequence in sequences] apparently tensor doen't works with strings
        print(labels)
        inputs = self.tokenizer(text=texts,
                                return_tensors='pt',
                                padding=True,
                                truncation=True,
                                max_length=self.max_seq_len)
        inputs.update({'labels': torch.tensor(labels)})

        return inputs
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2classificationcollator = Gpt2ClassificationCollator(tokenizer=tokenizer,
                                                        max_seq_len=60)
val_dataloader = DataLoader(dataset=val_dataset,
                            batch_size=32,
                            shuffle=False,
                            collate_fn=gpt2classificationcollator)

In [46]:
# #Get the tokenizer and model
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# model = GPT2LMHeadModel.from_pretrained('gpt2')

load_model_parameters = False
model_path = '/content/drive/My Drive/project_calculations/generative_model.pth'
if os.path.exists(model_path) and load_model_parameters:
  print("loading last model parameters..")
  model.load_state_dict(torch.load(model_path))

#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [47]:
from sklearn.metrics import accuracy_score
import time

def train(
    train_dataset, val_dataloader, model, tokenizer,
    batch_size=16, epochs=2, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 100

    device = 'cuda' if torch.cuda.is_available() else 'cpu'


    # device=torch.device("cuda")
    # model = model.cuda()
    model.to(device)
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None
    best_val_acc = 0
    all_loss = {'train_loss': [], 'val_loss': []}
    all_acc = {'train_acc': [], 'val_acc': []}

    for epoch in range(epochs):
      start_time = time.time()  # Start the timer
      prediction_labels = []
      true_labels = []
      total_train_loss = []
      model.train()
      for idx, entry in tqdm(enumerate(train_dataloader)):
          (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

          if carry_on and idx != len(train_dataloader) - 1:
              continue

          input_tensor = input_tensor.to(device)
          outputs = model(input_tensor, labels=input_tensor)
          loss, logits = outputs[:2]
          total_train_loss.append(loss.item())
          # total_train_loss.append(loss)
          loss.backward()

          if (accumulating_batch_count % batch_size) == 0:
              optimizer.step()
              scheduler.step()
              optimizer.zero_grad()
              model.zero_grad()

          accumulating_batch_count += 1
          input_tensor = None
          prediction_labels += logits.argmax(axis=-1).flatten().tolist()

      all_loss['train_loss'].append(total_train_loss)



      model.eval()
      prediction_labels = []
      true_labels = []
      val_total_loss = []

      for batch in val_dataloader:
          true_labels += batch['labels'].numpy().flatten().tolist()
          batch = {k:v.type(torch.long).to(device) for k, v in batch.items()}

          with torch.no_grad():
              outputs = model(**batch)
              loss, logits = outputs[:2]
              logits = logits.detach().cpu().numpy()
              val_total_loss.append(loss.item())

              prediction_labels += logits.argmax(axis=-1).flatten().tolist()
      val_acc = accuracy_score(true_labels, prediction_labels )
      all_acc['val_acc'].append(val_acc)
      all_loss['val_loss'].append(val_total_loss)
      if save_model_on_epoch and best_val_acc < val_acc:
        best_val_acc = val_acc
        print("bset best_val_acc so far is ", best_val_acc)
        torch.save(model.state_dict(), '/content/drive/My Drive/project_calculations/section_c_generative_model.pth')
      end_time = time.time()  # Stop the timer
      epoch_duration = end_time - start_time
      print(f"Training epoch {epoch} , Train Loss: {torch.tensor(total_train_loss).mean():.3f}, Val Acc: {val_acc:.3f}, Epoch Duration: {epoch_duration:.3f} seconds")

    return model

In [48]:
model = train(train_dataset=train_dataset, val_dataloader=val_dataloader, model=model, tokenizer=tokenizer, save_model_on_epoch=True, epochs=1)

100it [00:02, 46.72it/s]


[1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1]


ValueError: ignored